## Step1 : Extracting data from the text files.

In [55]:
import warnings
from bs4 import BeautifulSoup
from pathlib import Path
import pandas as pd
import re
warnings.simplefilter(action='ignore', category=FutureWarning)

In [56]:
### Create the necessary dataframes for extracting features

def initialize_df():

    global features, maping_description, maping_additional

    features = {
        'name': None,
        'model': None,
        'price': None,
        'color': None,
        'fuel_type': None,
        'origin_car': None,
        'car_license': None,
        'lime_type': None,
        'glass': None,
        'motor_power': None,
        'car_speedometer': None,
        'passengers': None,
        'ex_owners': None,
        'alarm_devise':None,
        'air_conditioner':None,
        'CD_recorder':None,
        'sunroof':None,
        'leather_upholstery':None,
        'central_closing':None,
        'magnesium_rims':None,
        'air_cushion':None,
        }
    maping_description = {
        "لون السيارة": 'color',
        "نوع الوقود": 'fuel_type',
        "أصل السيارة": 'origin_car',
        "عداد السيارة": 'car_speedometer',
        "أصحاب سابقون": 'ex_owners',
        "رخصة السيارة": 'car_license',
        "نوع الجير": 'lime_type',
        "الزجاج": 'glass',
        "قوة الماتور": 'motor_power',
        "عدد الركاب": 'passengers',
    }
    maping_additional={
        'جهاز إنذار': 'alarm_devise',
        'مُكيّف': 'air_conditioner',
        'مسجل CD': 'CD_recorder',
        'فتحة سقف': 'sunroof',
        'فرش جلد': 'leather_upholstery',
        'إغلاق مركزي': 'central_closing',
        'جنطات مغنيسيوم': 'magnesium_rims',
        'وسادة حماية هوائية': 'air_cushion',
    }

In [57]:
### Extracting model year features from file to features dictionary.


def get_model(model_element):

    for row in model_element:
        subitem = str(row.text).split()
        for item in subitem:
            if(item.isdigit()):
                features['model'] = item
                break

In [79]:
### Extracting price feature from file to features dictionary.


def get_price(price_element):
    if price_element:
        price = str(price_element.text)
        for subitem in price.split():
            if subitem.isdigit():
                features['price'] = int(subitem)
                break
    else:
        features['price'] = 0  # Set a default value or handle the case where price_element is not found


In [80]:
### Extracting description features from file to features dictionary.

def get_description(description_element):
    if description_element:
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", category=DeprecationWarning)
            for arabic_feature in maping_description.keys():
                description = description_element.find(string=arabic_feature)
                features[maping_description[arabic_feature]] = (
                    None if description is None else description.next_element.get_text()
                )
        
        

In [81]:

            
def get_additional_info(additional_element):
    if additional_element:
        for row in additional_element:
            row = row.text
            if row in maping_additional.keys():
                features[maping_additional[row]] = 1

In [82]:
# Create dataframe to add the data collected
data = pd.DataFrame()

for path in Path('data/').glob('*.txt'):
    initialize_df()
    with path.open(encoding='utf-8') as file_path:
        # create BeautifulSoup object for file by html parser
        soup = BeautifulSoup(file_path, "html.parser")

        # Extracting name value to features dictionary
        features['name'] = soup.find('h3', class_=None).text

        # Extracting model value to features dictionary
        get_model(soup.find('h5', class_=None))

        # Extracting price value to features dictionary
        get_price(soup.find('h5', class_='post-price'))

        # Extracting description values to features dictionary
        get_description(soup.find('table', class_='list_ads'))

        
        # Extracting additional values to features dictionary
        additional_info_element = soup.find('td', class_='list-additions')
        get_additional_info(additional_info_element.find_all('li') if additional_info_element else [])
        
        # Convert the dictionary to series
        sample = pd.Series(features)

        # Concating the sample series to the dataframe
        data = pd.concat((data, sample), axis=1, ignore_index=True)

In [83]:
data = data.T
data

,name,model,price,color,fuel_type,origin_car,car_license,lime_type,glass,motor_power,...,passengers,ex_owners,alarm_devise,air_conditioner,CD_recorder,sunroof,leather_upholstery,central_closing,magnesium_rims,air_cushion
0,كيا برايد,2008,44000,أسود,بنزين,خصوصي,فلسطينية,اوتوماتيك,الكتروني,1600,...,None,يد اولى,1,1,1,None,1,1,None,1
1,هونداي اكسنت,2013,58000,فضي,بنزين,خصوصي,فلسطينية,اوتوماتيك,الكتروني,1400,...,4+1,يد اولى,1,1,None,None,1,1,1,1
2,بيجو 508,2013,40000,أسود,بنزين,خصوصي,فلسطينية,نصف اوتوماتيك,الكتروني,2000,...,4+1,يد ثانيه,1,1,1,1,1,1,1,1
3,سكودا اوكتافيا,2010,70000,أزرق,ديزل,خصوصي,فلسطينية,عادي,الكتروني,1900,...,4+1,3,1,1,1,None,1,1,1,1
4,سوبارو امبريزا,1999,33000,رصاصي,بنزين,خصوصي,فلسطينية,عادي,الكتروني,1600,...,4+1,None,1,1,1,None,None,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7772,فيات فيات 132,1973,27000,أحمر,بنزين,خصوصي,فلسطينية,عادي,يدوي,1800,...,None,None,None,None,1,None,1,None,None,None
7773,هونداي سانتافيه,2016,117000,أسود,ديزل,خصوصي,فلسطينية,اوتوماتيك,الكتروني,2000,...,6+1,None,1,1,None,1,1,1,1,1
7774,هونداي اكسنت,2017,67000,فضي,بنزين,خصوصي,فلسطينية,اوتوماتيك,الكتروني,1400,...,4+1,1,1,1,1,None,1,1,1,1
7775,سكودا فابيا,2015,58000,أبيض,بنزين,خصوصي,فلسطينية,اوتوماتيك,الكتروني,1200,...,4+1,2,1,1,1,None,None,1,1,1


### Solve "معرض السيارات" error from extracting data 

In [84]:
count_matching_rows = len(data[data['name'] == 'معرض السيارات'])
print(f"Number of rows where name is 'معرض السيارات': {count_matching_rows}")


Number of rows where name is 'معرض السيارات': 913


In [85]:
data = data[data['name'] != 'معرض السيارات']

In [86]:
data

,name,model,price,color,fuel_type,origin_car,car_license,lime_type,glass,motor_power,...,passengers,ex_owners,alarm_devise,air_conditioner,CD_recorder,sunroof,leather_upholstery,central_closing,magnesium_rims,air_cushion
0,كيا برايد,2008,44000,أسود,بنزين,خصوصي,فلسطينية,اوتوماتيك,الكتروني,1600,...,None,يد اولى,1,1,1,None,1,1,None,1
1,هونداي اكسنت,2013,58000,فضي,بنزين,خصوصي,فلسطينية,اوتوماتيك,الكتروني,1400,...,4+1,يد اولى,1,1,None,None,1,1,1,1
2,بيجو 508,2013,40000,أسود,بنزين,خصوصي,فلسطينية,نصف اوتوماتيك,الكتروني,2000,...,4+1,يد ثانيه,1,1,1,1,1,1,1,1
3,سكودا اوكتافيا,2010,70000,أزرق,ديزل,خصوصي,فلسطينية,عادي,الكتروني,1900,...,4+1,3,1,1,1,None,1,1,1,1
4,سوبارو امبريزا,1999,33000,رصاصي,بنزين,خصوصي,فلسطينية,عادي,الكتروني,1600,...,4+1,None,1,1,1,None,None,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7772,فيات فيات 132,1973,27000,أحمر,بنزين,خصوصي,فلسطينية,عادي,يدوي,1800,...,None,None,None,None,1,None,1,None,None,None
7773,هونداي سانتافيه,2016,117000,أسود,ديزل,خصوصي,فلسطينية,اوتوماتيك,الكتروني,2000,...,6+1,None,1,1,None,1,1,1,1,1
7774,هونداي اكسنت,2017,67000,فضي,بنزين,خصوصي,فلسطينية,اوتوماتيك,الكتروني,1400,...,4+1,1,1,1,1,None,1,1,1,1
7775,سكودا فابيا,2015,58000,أبيض,بنزين,خصوصي,فلسطينية,اوتوماتيك,الكتروني,1200,...,4+1,2,1,1,1,None,None,1,1,1


In [87]:
data.shape

(6864, 21)

In [88]:
data.to_csv('data/data.csv')